# Convolutional Autoencoders
## Exercise 1

In [1]:
# Import libraries
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import plotly.express as px

In [2]:
def get_data():
    # Load data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_full = np.concatenate((x_train, x_test), axis=0)
    y_full = np.concatenate((y_train, y_test), axis=0)

    # Split data
    x_train, x_temp, y_train, y_temp = train_test_split(x_full, y_full, test_size=0.2, random_state=42)
    x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
    print(f"Training set shape: {x_train.shape}, {y_train.shape}")
    print(f"Validation set shape: {x_val.shape}, {y_val.shape}")
    print(f"Test set shape: {x_test.shape}, {y_test.shape}")

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_val = x_val.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_val = tf.keras.utils.to_categorical(y_val, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    #Return data
    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

In [13]:
def build_autoencoder(input_shape=(32, 32, 3)):
    # Build the autoencoder model
    autoencoder = tf.keras.models.Sequential([
        # Encoder
        tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
        tf.keras.layers.Conv2D(12, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.MaxPooling2D((2, 2), padding='same'),
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),

        # Decoder
        tf.keras.layers.UpSampling2D((2, 2)),
        tf.keras.layers.Conv2D(12, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.UpSampling2D((2, 2)),
        tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')
    ])

    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.summary()
    
    return autoencoder

In [4]:
def train_autoencoder(autoencoder, x_train, x_val):
    # Train the autoencoder
    train_history = autoencoder.fit(x_train, x_train,
                    epochs=10,
                    batch_size=256,
                    shuffle=True,
                    validation_data=(x_val, x_val))
    
    return train_history


In [5]:
def evaluate_autoencoder(autoencoder, x_test):
    # Evaluate the autoencoder
    test_loss = autoencoder.evaluate(x_test, x_test)
    print(f'Test Loss: {test_loss}')

    return test_loss

In [6]:
def plot_training_history(train_history):
    # Plot training & validation loss values
    fig = px.line(x=train_history.epoch, 
                  y=[train_history.history['loss'], train_history.history['val_loss']],
                  labels={'x': 'Epoch', 'y': 'Loss'},
                  title='Model Loss')
    fig.update_layout(legend_title_text='Legend')
    fig.data[0].name = 'Train Loss'
    fig.data[1].name = 'Validation Loss'
    fig.show()

In [7]:
# Load and preprocess data
(x_train, y_train), (x_val, y_val), (x_test, y_test) = get_data()

Training set shape: (48000, 32, 32, 3), (48000, 1)
Validation set shape: (6000, 32, 32, 3), (6000, 1)
Test set shape: (6000, 32, 32, 3), (6000, 1)


In [8]:
# Build the autoencoder
autoencoder = build_autoencoder()

d:\OneDrive\Documentos\UM\um_code\ACML\assignments\autoencoders\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 8)      │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 12)     │           876 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 12)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 16)       │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 12)     │         1,740 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 32, 32, 12)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 3)      │           327 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,911 (19.18 KB)

 Trainable params: 4,911 (19.18 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Train the autoencoder
train_history = train_autoencoder(autoencoder, x_train, x_val)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - loss: 0.0234 - val_loss: 0.0123
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 0.0106 - val_loss: 0.0094
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 0.0088 - val_loss: 0.0083
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.0079 - val_loss: 0.0076
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0070 - val_loss: 0.0069
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0067 - val_loss: 0.0066
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0065 - val_loss: 0.0065
Epoch 9/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0063 - val_loss: 0.0065
Epoch 10/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.0062 - val_loss: 0.0061


In [10]:
# Evaluate the autoencoder
test_loss = evaluate_autoencoder(autoencoder, x_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0061
Test Loss: 0.0061337812803685665


In [11]:
# Plot training history
plot_training_history(train_history)

## Exercise 3

In [ ]:
def rgb_to_gray(images):
    """Convert RGB [N,32,32,3] → Grayscale [N,32,32,1]."""
    return np.dot(images[...,:3], [0.299, 0.587, 0.114])[..., np.newaxis]

x_train_gray = rgb_to_gray(x_train)
x_val_gray   = rgb_to_gray(x_val)
x_test_gray  = rgb_to_gray(x_test)

Training data: (48000, 32, 32, 1) -> (48000, 32, 32, 3)


In [15]:
# Build the autoencoder
autoencoder_gray_color = build_autoencoder(input_shape=(32, 32, 1))

d:\OneDrive\Documentos\UM\um_code\ACML\assignments\autoencoders\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 8)      │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 12)     │           876 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 12)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 16)       │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 16, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 12)     │         1,740 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 32, 32, 12)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 3)      │           327 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,767 (18.62 KB)

 Trainable params: 4,767 (18.62 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the autoencoder
train_history = train_autoencoder(autoencoder_gray_color, x_train_gray, x_val_gray)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - loss: 0.0194 - val_loss: 0.0096
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0082 - val_loss: 0.0073
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0068 - val_loss: 0.0064
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - loss: 0.0062 - val_loss: 0.0060
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0058 - val_loss: 0.0056
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0055 - val_loss: 0.0054
Epoch 7/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - loss: 0.0053 - val_loss: 0.0053
Epoch 8/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0052 - val_loss: 0.0051
Epoch 9/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0050 - val_loss: 0.0051
Epoch 10/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 0.0049 - val_loss: 0.0049


In [14]:
# Evaluate the autoencoder
test_loss = evaluate_autoencoder(autoencoder, x_test)

188/188 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0061
Test Loss: 0.0061337812803685665


In [17]:
# Plot training history
plot_training_history(train_history)